In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as pp
import xml.etree.ElementTree as ET
import pandas as pd
import os, sys, glob

from cycler import cycler
EP = '{http://www.molpro.net/schema/molpro-output}' #element prefix, somehow gets added
%matplotlib widget

## Define functions

* **`get_correlation_energy`**

In [ ]:
def get_correlation_energy(root, calc, return_total=False):
    """
    Get correlation energy for a job, given a calc type

    Arguments:
        root: <xml.etree.ElementTree.Element> obtained from parsing an output xml file
        calc: <str> method name to look for in job attributes.
        
    Returns (if return_total is False):
        correlation_energy (float) 
    (if return_total is True):
        correlation_energy, total_energy
    """

    for jobstep in root.iter(f'{EP}jobstep'):
        if jobstep.attrib['command'] == calc:
            desired_jobstep = jobstep
            break

    for prop in jobstep.iter(f'{EP}property'):
        if prop.attrib['method'] == 'Reference':
            ref_ener = float(prop.attrib['value'])
        if prop.attrib['name'] == 'total energy' and prop.attrib['method'] == calc:
            total_ener = float(prop.attrib['value'])
  
    correlation_energy = total_ener - ref_ener
    if return_total:
        return correlation_energy, total_ener

    return correlation_energy

def get_all_correlation_eneriges(roots, calc, key='correlation_energies', data=None):
    """
    Get correlation energies for all bases.

    Arguments:
        roots: a dictionary of {basis: root, ...}
    returns basis, cener, aug-cener
    """
    if data is None:
        data = dict(
            basis_names = roots.keys(),
            basis_ints = np.arange(2., len(roots) + 2),
        )
    data[key] = []
        
    for basis, root in roots.items():
        cener = get_correlation_energy(root, calc)
        data[key].append(cener)
        
    return data
    
def getroots(bases, prefix='neon_', outdir='outputs', skipmissing=True):
    '''Get a dictionary of XML trees for each basis'''
    roots = dict()
    
    for basis in bases:
        search_str = f'{outdir}/{prefix}{basis}.*.xml'
        fname_matches = glob.glob(search_str)
        fname = fname_matches[-1]
        #print(fname)
        try:
            root = ET.parse(fname).getroot()
        except FileNotFoundError as e:
            if skipmissing:
                print(f"{fname} not found, skipping")
            else:
                raise e
        if not list(root.iter(f'{EP}error')):
            roots[basis] = root
        else:
            print(f"Error found in {fname}, skipping")
    return roots


## Define basis

In [ ]:
bases = 'VDZ VTZ VQZ V5Z V6Z'.split()
bases = [f'cc-p{b}' for b in bases]
bases_ints = np.arange(2., len(bases) + 2)


## Main starts here

In [ ]:


def main(calc, calc_type, ccsd=True, ccp=True, aug=True, ax=None, label=None, write=False):
    outdir=f'outputs/neon_{calc_type}/'
    roots = getroots(
        bases, prefix=f'neon_{calc_type}_', outdir=outdir)

   
    roots_aug = getroots(
        bases, prefix=f'neon_{calc_type}_aug-', outdir=outdir)
        
    
    data = get_all_correlation_eneriges(roots, calc)
    if ccsd:
        data = get_all_correlation_eneriges(roots, 'CCSD', key='CCSD_corener', data=data)
    if aug:
        data = get_all_correlation_eneriges(
            roots_aug, calc, key='correlation_energies_aug', data=data
        )

    if write:
        df = pd.DataFrame(data)
        df.to_csv(f'{outdir}/data.csv')
        
    if ax is None:
        _, ax = pp.subplots()

    if label is None:
        label_ccp = 'cc-p*'
        label_aug = 'aug-cc-p*'
        label_ccsd = 'cc-p* (ccsd)'
    else:
        label_ccp = label_aug = label_ccsd = label
        
    inv_x3 = data['basis_ints'] ** -4
    if ccp:
        ax.plot(
            inv_x3, data['correlation_energies'], 
            marker='o', label=label_ccp,
        )

    if aug:
        ax.plot(
            inv_x3, data['correlation_energies_aug'], 
            marker='^', label=label_aug,
        )
    if ccsd:
        ax.plot(
            inv_x3, data['CCSD_corener'],
            #linestyle='--',
            marker='+', label=label_ccsd,
        )
    
    #'''
    ax.set_ylabel('Energy (hartree)')
    ax.set_xlabel('$X^{-3}$')
    ax.set_title(f"{calc_type} | {calc}")
    #ax.legend()

    return roots, roots_aug, data, ax


In [ ]:

calculations = (
    ('hf_ccsdt', 'CCSD(T)', True),
    ('df-hf_df-ccsdt', 'CCSD(T)', True),
    ('hf_mp2', 'MP2', False),
    ('df-hf_df-ccsdt-f12', 'CCSD(T)-F12', True),
    ('df-hf_df-ccsdt-f12', 'MP2-F12', False),
)

num_cols = 2
num_rows = (len(calculations) + (num_cols - 1)) // num_cols

fig, axes = pp.subplots(num_rows, num_cols, sharey=True, sharex=True)
flat_axes = axes.flatten()
fig.subplots_adjust(wspace= 0)
fig.set_size_inches(8,12)

df_dict = dict(
    calc_type = [], 
    ccpV5Z=[], 
    ccpV_extr=[],
    aug_ccpV5Z=[],
    aug_ccpV_extr=[],
    ccsd_ccpV5Z=[],
    ccsd_ccpV_extr=[],
    lowest_extrapolated=[]
)
plot_2_dict_label = {
        'cc-p*': 'ccpV_extr',
        'aug-cc-p*':'aug_ccpV_extr',
        'cc-p* (ccsd)':'ccsd_ccpV_extr'
}
    
for n, (calc_type, calc, ccsd) in enumerate(calculations):
    #print(calc_type, calc, ccsd)
    ax = flat_axes[n]
    _, _, data, _ = main(calc, calc_type, ccsd=ccsd, ax=ax, write=True)
    df_dict['calc_type'].append( f"{calc_type}_{calc}")
    df_dict['ccpV5Z'].append(data['correlation_energies'][3])
    df_dict['aug_ccpV5Z'].append(data['correlation_energies_aug'][3])
    if ccsd:
        df_dict['ccsd_ccpV5Z'].append(data['CCSD_corener'][3])
    else:
        df_dict['ccsd_ccpV5Z'].append(np.nan)
        df_dict['ccsd_ccpV_extr'].append(np.nan)
        
    x = ax.lines[0].get_xdata()
    xx = np.linspace(0, max(x))
    y_intercept = 0
    best_calc = ''
    best_gradient = 0

    for line in ax.lines:
        
        y = line.get_ydata()
        gradient, y_i = np.polyfit(x[1:], y[1:], 1)
    
        extrapolated_line = gradient * xx + y_i
    
        ax.plot(xx, extrapolated_line, linewidth=2, linestyle='--', color=line.get_color())
        lab = line.get_label()
        dict_label = plot_2_dict_label[lab]

        df_dict[dict_label].append(y_i)
        
        if y_i < y_intercept:
            y_intercept = y_i
            best_gradient = gradient
            best_calc = lab

    df_dict['lowest_extrapolated'].append(best_calc)
    

def draw_references(line, ax, additional_points=None):
    color = line.get_color()
    ys = line.get_ydata()
    if additional_points is not None:
        ys = np.concatenate((ys, additional_points))
    for y in ys:
        ax.axhline(y, color=color, linestyle='dotted', linewidth=1)

show_ref = 'cc-p*'
for line in axes[0, 0].lines:
    
    if line.get_label() != show_ref: continue

    for ax in axes.flatten():
        draw_references(line, ax,)

ax = axes[0, 0]
ax.legend()
inv_X3 = ax.lines[0].get_xdata()
ax.set_xlim(0, inv_X3[1])
ax.set_ylim(None, ax.lines[0].get_ydata()[1])

In [ ]:
#mpl.rcParams['axes.prop_cycle'] = cycler(color='bcgrmyk')
mpl.rcParams['axes.prop_cycle'] = cycler('color', ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])
fig, axes = pp.subplots(1, 2, sharey=True, sharex=True)

fig.set_size_inches(10,6.5)
augs = (False, True)
for i, ax in enumerate(axes):
    ax.set_prop_cycle(None)
    for n, (calc_type, calc, ccsd) in enumerate(calculations):
        lab = f"{calc_type} | {calc}"
        main(calc, calc_type, ccsd=False, ccp=not augs[i], aug=augs[i], label=lab, ax=ax,)
    
        line = ax.lines[-1]
        x = line.get_xdata()
        xx = np.linspace(0, max(x))
        
        y = line.get_ydata()
        gradient, y_i = np.polyfit(x[1:], y[1:], 1)
    
        extrapolated_line = gradient * xx + y_i
        ax.plot(xx, extrapolated_line, linewidth=2, linestyle='--', color=line.get_color())
    
        if lab == 'hf_ccsdt | CCSD(T)':
            draw_references(line, ax, additional_points=[y_i])
        ax.set_title(augs[i]*"aug_" + 'cc-p* compared')
ax.legend()

In [ ]:
summary_df = pd.DataFrame(df_dict)
summary_df = summary_df.sort_values('ccpV_extr')
summary_df.ccpV_extr - summary_df.aug_ccpV_extr

In [ ]:
ax = summary_df.plot.barh(x='calc_type', y=['ccpV_extr', 'aug_ccpV_extr'])
ax.set_xlim(-0.330, -0.310)
fig = ax.get_figure()
fig.set_size_inches(8,3)
fig.tight_layout()

In [ ]:
fname = 'summary_of_lowest_correlation_energies.csv'
summary_df.to_csv(fname, index=False)

## [ Inspect a particular XML tree ]  ![] = optional cell

In [ ]:
root = ET.parse(
    'outputs/neon_hf_mp2/neon_hf_mp2_aug-cc-pV5Z.133732-20241030.xml'
).getroot()
for js in root.iter(f'{EP}jobstep'):
    js
for prop in js.iter(f'{EP}property'):
    name = prop.attrib['name']
    method = prop.attrib['method']
    print(f"{name} | {method}")
print(list(root.iter(f'{EP}property')))    
print(list(root.iter(f'{EP}error')))



In [ ]:
root = ET.parse(
    'outputs/neon_df-hf_df-ccsdt-f12/neon_df-hf_df-ccsdt-f12_cc-pVTZ.133769-20241031.xml'
).getroot()
for js in root.iter(f'{EP}jobstep'):
    js
for prop in js.iter(f'{EP}property'):
    name = prop.attrib['name']
    method = prop.attrib['method']
    print(f"{name} | {method}")
print(list(root.iter(f'{EP}property')))    
print(list(root.iter(f'{EP}error')))